In [3]:
import pandas as pd
import numpy as np

In [4]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [5]:
train["Age"] = train["Age"].fillna(train["Age"].median())

In [6]:
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])

In [7]:
train.loc[train["Sex"] == "female", "Sex"] = 1
train.loc[train["Sex"] == "male", "Sex"] = 0

In [8]:
def h(x):
    return 1/(1+np.exp(-x))

def dj_dtheta(theta,X,Y,j):
    result=0
    for i in range(np.size(Y)):
        error=Y[i]-h(np.dot(theta,X[i]))
        result+=(error*X[i,j])
    return result

def grad_des(X,Y,theta):
    lnr=0.0001
    epoch=50000
    for i in range(epoch):
        if(i%5000==0):
            print(i,theta)
        new_theta=[]
        for j in range(X.shape[1]):
            a=theta[j]+lnr*dj_dtheta(theta,X,Y,j)
            new_theta.append(a)
        theta=np.array(new_theta)
        
    return theta
    
    

In [9]:
data = np.array(train[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
data=np.insert(data, 0, 1, axis=1)
normalizer=max(data[:,[3]])
data[:,[3]]=data[:,[3]]/normalizer
add_fea=data[:,[3]]**2
data = np.hstack([data, add_fea])
print(data)

[[1.       3.       0.       0.275    0.       0.075625]
 [1.       1.       1.       0.475    1.       0.225625]
 [1.       3.       1.       0.325    0.       0.105625]
 ...
 [1.       3.       1.       0.35     0.       0.1225  ]
 [1.       1.       0.       0.325    1.       0.105625]
 [1.       3.       0.       0.4      2.       0.16    ]]


In [10]:
theta=np.array([0,0,0,0,0,0])
Y=np.array(train["Survived"].values, dtype = float)
theta=grad_des(data,Y,theta)
print("theta",theta)

0 [0 0 0 0 0 0]


5000 [ 1.53181579 -1.11426239  2.57154241 -1.37999538  0.31562105 -0.79408339]
10000 [ 1.8189545  -1.16872899  2.57254399 -1.85990669  0.31681973 -0.70861804]
15000 [ 1.90770441 -1.17964931  2.57319208 -2.14085136  0.31797215 -0.46933813]
20000 [ 1.95389937 -1.18162023  2.5733514  -2.3682205   0.31900998 -0.21454346]
25000 [ 1.98939758 -1.18178164  2.57347642 -2.57257397  0.31996418  0.02858727]
30000 [ 2.02093572 -1.18161829  2.573665   -2.76101588  0.32084694  0.25540566]
35000 [ 2.05001473 -1.18143187  2.57392259 -2.93573998  0.32166464  0.46590197]
40000 [ 2.07704132 -1.18127792  2.57423688 -3.09788499  0.32242217  0.66097718]
45000 [ 2.10219193 -1.18116173  2.574594   -3.24833605  0.32312384  0.84166655]
theta [ 2.12559195 -1.18107909  2.57498167 -3.38789082  0.32377366  1.00898005]


In [15]:
def isSurvived(theta,X,m):
    result=[]
    for i in range(m):
        ans=h(np.dot(theta,X[i]))
        if(ans>=0.5):
            ans=1
        else:
            ans=0
        result.append(ans)
    return result
        

dic=dict()
dic["PassengerId"]=np.array(test["PassengerId"].values, dtype = int)
test.loc[test["Sex"] == "female", "Sex"] = 1
test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Embarked"] == "S", "Embarked"] = 0
test.loc[test["Embarked"] == "C", "Embarked"] = 1
test.loc[test["Embarked"] == "Q", "Embarked"] = 2
data=np.array(test[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
data=np.insert(data, 0, 1, axis=1)
data[:,[3]]=data[:,[3]]/normalizer
add_fea=data[:,[3]]**2
data = np.hstack([data, add_fea])
dic["Survived"]=isSurvived(theta,data,418)
df=pd.DataFrame(dic)
df.to_csv("T12(ageXX2).csv",index=False)








In [12]:
def accuracy(theta,X,Y):
    count=0
    for i in range(np.size(Y)):
        predY=h(np.dot(theta,X[i]))
        if(predY>=0.5):
            predY=1
        else:
            predY=0
        
        if(predY==Y[i]):
            count+=1
    print(count/np.size(Y))
    



data = np.array(train[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
data=np.insert(data, 0, 1, axis=1)
normalizer=max(data[:,[3]])
data[:,[3]]=data[:,[3]]/normalizer
add_fea=data[:,[3]]*data[:,[1]]
data = np.hstack([data, add_fea])
Y=np.array(train["Survived"].values, dtype = float)
accuracy(theta,data,Y)
        

0.7755331088664422
